In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from preprocessing.create_dummy import create_dummy
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('../data/filled.csv')
data_to_model = create_dummy(df)

In [3]:
data_to_model = shuffle(data_to_model, random_state=0)
X = data_to_model.drop(columns="BAD")
y = data_to_model['BAD']

In [4]:
for col in X.columns:
    min_ = X[col].min()
    max_ = X[col].max()
    X[col] -= min_
    X[col] /= max_

In [5]:
X

,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self
916,0.087875,0.206405,0.112161,0.073171,0.0,0.000000,0.043370,0.058824,0.042254,0.163568,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3338,0.184650,0.177319,0.104353,0.024390,0.0,0.000000,0.070532,0.000000,0.239437,0.122003,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5754,0.473860,0.179446,0.047359,0.217616,0.6,0.000000,0.081936,0.069768,0.267606,0.130128,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4290,0.236930,0.240708,0.149036,0.217616,0.0,0.000000,0.166304,0.000000,0.281690,0.141896,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1540,0.113459,0.110267,0.052472,0.463415,0.0,0.000000,0.196400,0.058824,0.239437,0.176135,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,0.276974,0.552166,0.325132,0.146341,0.0,0.266667,0.415966,0.069768,0.774648,0.160910,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3264,0.181313,0.279882,0.148551,0.390244,0.0,0.000000,0.149030,0.058824,0.239437,0.182523,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1653,0.119021,0.139399,0.067221,0.268293,0.0,0.000000,0.092849,0.058824,0.253521,0.158667,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2607,0.154616,0.192559,0.116835,0.009756,0.0,0.000000,0.329985,0.176471,0.436620,0.163568,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
import random
# actions GOOD BAD
GOOD = 0
BAD = 1

class EnvironmentSimulator:
    def __init__(self, labels=y):
        self.at_eq_yt = 1  # payout for a_t == y_t
        self.at_ne_yt = -1  # payout for a_t != y_t
        self.y_t = labels.tolist()
        self.n = len(self.y_t)

    def take_action(self, state, action):
        # compare a_t with actual label y_t
        if action == BAD and self.y_t[state] == 1:
            reward = 1
        elif action == GOOD and self.y_t[state] == 0:
            reward = 1/4
        elif action == GOOD and self.y_t[state] == 1:  # a_t != y_t
            reward = -1
        else: # action == BAD and self.y_t[state] == 0
            reward = -1/4

        return reward


In [7]:
import random
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()


class Agent:
    def __init__(self, learning_rate=0.5, discount=0.95, exploration_rate=1.0, iterations=10000, data=X):
        self.learning_rate = learning_rate
        self.discount = discount # How much we appreciate future reward over current
        self.exploration_rate = exploration_rate # Initial exploration rate
        self.exploration_delta = 1.0 / iterations # Shift from exploration to explotation

        # Input has 12 neuron representing new client in our state
        self.input_count = 18
        # Output is two neurons, each represents Q-value for action (GOOD and BAD)
        self.output_count = 2
        self.data = data

        self.session = tf.Session()
        self.define_model()
        self.session.run(self.initializer)

    # Define tensorflow model graph
    def define_model(self):
        # Input is an array of single item (state)
        self.model_input = tf.placeholder(dtype=tf.float32, shape=[None, self.input_count])

        # 32 hidden neurons per layer
        layer_size = 1024
        # Two hidden layers of 32 neurons with sigmoid activation initialized to zero for stability
        fc1 = tf.layers.dense(self.model_input, layer_size, activation='relu', kernel_initializer='glorot_uniform')
        fc2 = tf.layers.dense(fc1, layer_size, activation='relu', kernel_initializer='glorot_uniform')
        fc3 = tf.layers.dense(fc2, layer_size, activation='relu', kernel_initializer='glorot_uniform')
        fc4 = tf.layers.dense(fc3, layer_size, activation='relu', kernel_initializer='glorot_uniform')

        # Output is two values, Q for both possible actions GOOD and BAD
        self.model_output = tf.layers.dense(fc4, self.output_count)

        # This is for feeding training output (a.k.a ideal target values)
        self.target_output = tf.placeholder(shape=[None, self.output_count], dtype=tf.float32)
        # Loss is mean squared difference between current output and ideal target values
        loss = tf.losses.compute_weighted_loss(self.target_output, self.model_output)
        # Optimizer adjusts weights to minimize loss, with the speed of learning_rate
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(loss)
        # Initializer to set weights to initial values
        self.initializer = tf.global_variables_initializer()

    # Ask model to estimate Q value for specific state (inference)
    def get_Q(self, state):
        # Model input: Single state represented by array of single item (state)
        # Model output: Array of Q values for single state
        return self.session.run(self.model_output, feed_dict={self.model_input: self.data.iloc[state].values.reshape((-1, 18))})[0]

    def get_next_action(self, state):
        # if random.random() > self.exploration_rate:
        return self.greedy_action(state)
        # else:
        #     return self.random_action()

    # Which action (GOOD or BAD) has bigger Q-value, estimated by our model (inference).
    def greedy_action(self, state):
        # argmax picks the higher Q-value and returns the index (GOOD=0, BAD=1)
        return np.argmax(self.get_Q(state))

    def random_action(self):
        return GOOD if random.random() < 0.5 else BAD

    def train(self, old_state, action, reward, new_state):
        # Ask the model for the Q values of the old state (inference)
        old_state_Q_values = self.get_Q(old_state)

        # Ask the model for the Q values of the new state (inference)
        new_state_Q_values = self.get_Q(new_state)

        # Real Q value for the action we took. This is what we will train towards.
        old_state_Q_values[action] = reward + self.discount * np.amax(new_state_Q_values)

        # Setup training data
        training_input = self.data.iloc[old_state].values.reshape((-1, 18))

        target_output = [old_state_Q_values]
        training_data = {self.model_input: training_input, self.target_output: target_output}

        # Train
        self.session.run(self.optimizer, feed_dict=training_data)

    def update(self, old_state, new_state, action, reward):
        # Train our model with new data
        self.train(old_state, action, reward, new_state)

        # Finally shift our exploration_rate toward zero (less gambling)
        if self.exploration_rate > 0:
            self.exploration_rate -= self.exploration_delta

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
import random
import json

iters = 100000
agent = Agent(learning_rate=1e-4, discount=0.1, iterations=iters)

# setup simulation
environment = EnvironmentSimulator(labels=y)
# environment.reset()
total_reward = 0 # Score keeping
last_total = 0

train_inds = X.index[:round(0.7 * X.shape[0])]
test_inds = X.index[round(0.7 * X.shape[0]):]

# train
for step in range(iters):
    old_state = np.random.choice(train_inds, 1)[0] # Store current state
    action = agent.get_next_action(old_state) # Query agent for the next action
    reward = environment.take_action(state=old_state, action=action) # Take action, get new state and reward
    new_state = np.random.choice(train_inds, 1)[0]
    agent.update(old_state, new_state, action, reward) # Let the agent update internals

    total_reward += reward # Keep score
    if step % 250 == 0: # Print out metadata every 250th iteration
        performance = (total_reward - last_total) / 250.0
        print(json.dumps({'step': step, 'performance': performance, 'total_reward': total_reward}))
        last_total = total_reward


# if __name__ == "__main__":
#     main()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
{"step": 0, "performance": 0.001, "total_reward": 0.25}
{"step": 250, "performance": -0.027, "total_reward": -6.5}
{"step": 500, "performance": -0.049, "total_reward": -18.75}
{"step": 750, "performance": -0.029, "total_reward": -26.0}
{"step": 1000, "performance": 0.019, "total_reward": -21.25}
{"step": 1250, "performance": -0.032, "total_reward": -29.25}
{"step": 1500, "performance": -0.045, "total_reward": -40.5}
{"step": 1750, "performance": -0.104, "total_reward": -66.5}
{"step": 2000, "performance": -0.101, "total_reward": -91.75}
{"step": 2250, "performance": -0.124, "total_reward": -122.75}
{"step": 2500, "performance": -0.141, "total_reward": -158.0}
{"step": 2750, "performance": -0.083, "total_reward": -178.75}
{"step": 3000, "performance": -0.027, "total_reward": -185.5}
{"step": 3250, "performance": -0.029, "total_reward": -192.75}
{"step": 3500,

In [9]:
y_pred = []
for ind in test_inds:
    y_pred.append(agent.get_next_action(ind))

In [10]:
y_test = y.iloc[test_inds]
y_train = y.iloc[train_inds]

In [11]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
print("Accuracy score : ", accuracy_score(y_train, y_pred))
print("F1 score : ", f1_score(y_train, y_pred))
# print("Precision score : ", precision_score(y_train, y_pred))
# print("Recall score : ", recall_score(y_train, y_pred))
print('ROC-AUC: ', roc_auc_score(y_test, y_pred))

Accuracy score :  0.8998476755056
F1 score :  0.825348957833034
ROC-AUC:  0.8183675505694373
